# Chapter 2 — TensorFlow 2
Detailed, practical overview of TensorFlow 2: execution model, building blocks, and common neural-network operations. Figures included inline.

## 2.1 First steps with TensorFlow 2

This chapter demonstrates how TensorFlow 2 is designed for both **easy development** (eager execution) and **high-performance production** (graph tracing / compilation). We'll start with a simple Multilayer Perceptron (MLP) example to ground the discussion, then expand into the core concepts that make TensorFlow work under the hood.

### 2.1.1 A simple MLP (concept)
An MLP (fully connected network) has input, hidden, and output layers. For a single hidden layer:
- Hidden activation:  $h = \sigma(x W_1 + b_1)$
- Output:  $y = \mathrm{softmax}(h W_2 + b_2)$

Here $\sigma$ is a nonlinear activation (sigmoid, ReLU, etc.). The MLP illustrates the common workflow: data → linear transform → nonlinearity → output normalization.


### Figure 2.1 — MLP diagram
<p align='left'><img src="./figure/figure2.1.png" width="60%"></p>
_Figure 2.1 visualizes inputs, weights, biases, hidden units and softmax outputs._

## 2.1.2 Eager execution vs. graph tracing
TensorFlow 2 runs in **eager mode** by default — operations execute immediately and return concrete values, making debugging and iteration simple. But for performance, TensorFlow provides `@tf.function` to trace a Python function and compile it into an optimized data-flow graph. The lifecycle looks like:
1. **First call**: trace the Python function and build a graph (this can be expensive).
2. **Optimize & place ops** on devices (CPU/GPU/TPU).
3. **Subsequent calls**: reuse the compiled graph, gaining significant speed.

Use `@tf.function` when your function runs many iterations or heavy numeric work; avoid tracing for one-off light operations.

### `@tf.function` and AutoGraph (example)
AutoGraph converts Python control flow into graph constructs. Here's a minimal example:
```python
import tensorflow as tf

@tf.function
def forward(x, W, b):
    return tf.matmul(x, W) + b

# tracing happens on first call
x = tf.constant([[1.0, 2.0]])
W = tf.constant([[0.5],[0.2]])
b = tf.constant([0.1])
print(forward(x, W, b))
```
**Caveats:** AutoGraph will convert Python lists/NumPy arrays to `tf.constant` and may unroll loops—be careful with very large or variable Python-side structures.

### Figure 2.5 — Tracing + Execution flow
<p align='left'><img src="./figure/figure2.5.png" width="60%"></p>
_Figure 2.5 illustrates tracing on the first call and feeding inputs on subsequent calls._

## 2.2 TensorFlow building blocks
Everything in TensorFlow is constructed from three primitives: `tf.Variable`, `tf.Tensor`, and `tf.Operation`. Understanding these is essential even when you use higher-level APIs such as Keras.

### tf.Variable (mutable state)
- `tf.Variable` holds parameters that change during training (weights and biases).
- Variables expose assign/update methods and interact with optimizers.

Example:
```python
import tensorflow as tf
W = tf.Variable(tf.random.normal([4, 3]))  # learnable weights
b = tf.Variable(tf.zeros([3]))
```

### tf.Tensor (immutable values)
- `tf.Tensor` represents concrete (read-only) data produced by operations.
- Use tensors for model inputs, intermediate activations, and constants.

Example:
```python
x = tf.constant([[1.0, 2.0, 3.0, 4.0]])
```

### tf.Operation (computations)
- TensorFlow ops perform transformations: `tf.matmul`, `tf.nn.conv2d`, `tf.add`.
- Composing ops yields computation graphs that can be optimized and executed efficiently on accelerators.


### Figure 2.4 — Example computational graph
<p align='left'><img src="./figure/figure2.4.png" width="60%"></p>
_This figure shows nodes (ops) and tensors flowing between them; higher-level Keras layers are built on these primitives._

## 2.3 Neural network–related computations
This section describes the key ops used in neural networks: matrix multiplication, convolution, and pooling. These are heavily optimized and form the core of NN performance.

### 2.3.1 Matrix multiplication (`tf.matmul`)
- Dense layers are implemented with matrix multiply followed by bias addition and activation.
- TensorFlow calls optimized BLAS/cuBLAS kernels on CPU/GPU for best throughput.

Snippet:
```python
x = tf.random.normal([batch_size, input_dim])
W = tf.Variable(tf.random.normal([input_dim, output_dim]))
b = tf.Variable(tf.zeros([output_dim]))
out = tf.matmul(x, W) + b
```

### 2.3.2 Convolutions (`tf.nn.conv2d` / `tf.nn.convolution`)
Convolution is central to CNNs. Key parameters:
- Input shape: `[batch, height, width, channels]`
- Kernel shape: `[k_h, k_w, in_ch, out_ch]`
- Strides and padding (`'SAME'` vs `'VALID'`).

Example:
```python
x = tf.random.normal([1, 28, 28, 3])
kernel = tf.random.normal([5, 5, 3, 32])
conv = tf.nn.conv2d(x, kernel, strides=[1,1,1,1], padding='SAME')
```
The chapter includes exercises that show how to reshape inputs and compute output shapes manually.

### Figure 2.2 — Convolution geometry
<p align='left'><img src="./figure/figure2.2.png" width="60%"></p>
_Shows kernel sliding over input, stride effect and padding choices._

### 2.3.3 Pooling operations (`tf.nn.max_pool2d`, `tf.nn.avg_pool2d`)
- Pooling down-samples spatial dimensions and helps make features invariant to small translations.
Example:
```python
pooled = tf.nn.max_pool2d(conv, ksize=2, strides=2, padding='VALID')
```

### Figure 2.3 — Pooling result comparison
<p align='left'><img src="./figure/figure2.3.png" width="60%"></p>
_Visual comparison of original feature map, average pooling and max pooling outputs._

## 2.4 Practical considerations and AutoGraph caveats
AutoGraph helps convert Python control flow into graph ops, but be mindful:
- **Tracing overhead:** functions traced only once; tracing can be expensive if the function is run rarely.
- **Implicit conversions:** Python lists/NumPy arrays inside traced functions convert to `tf.constant`.
- **Large unrolled loops:** can create enormous graphs—avoid unrolling very large loops inside traced functions.

Best practice: keep traced functions focused on numeric ops; move data preparation or Python-only logic outside the traced function. Use `tf.autograph.to_graph` docs for advanced scenarios.

### Figure 2.6 — AutoGraph example (control flow conversion)
<p align='left'><img src="./figure/figure2.6.png" width="60%"></p>
_Illustrates how an `if`/`for` in Python maps to graph constructs like `tf.cond` and `tf.while_loop`._

## 2.5 Additional useful utilities
TensorFlow provides many helper libraries and tools referenced in this chapter:
- **TensorFlow Datasets (tfds)** — ready-made datasets & preprocessing pipelines.
- **TensorBoard** — visualize training metrics, computational graphs and profiling information.
- **TensorFlow Hub** — repository of reusable pretrained modules.
- **Estimator API** — opinionated API useful in robust production workflows (less flexible than Keras, but safer).

Figure 2.7 and 2.8 show example TensorBoard views and TF Hub usage diagrams.

### Figure 2.7 — TensorBoard example
<p align='left'><img src="./figure/figure2.7.png" width="60%"></p>
### Figure 2.8 — TensorFlow Hub concept
<p align='left'><img src="./figure/figure2.8.png" width="60%"></p>

## 2.6 Exercises & quick snippets
A few short exercises or code snippets to solidify understanding are included in the book. Two quick runnable examples below:

1) **Compute an MLP forward pass (eager)**
```python
import tensorflow as tf
x = tf.constant([[1.,2.,3.,4.]])
W1 = tf.random.normal([4,3])
b1 = tf.zeros([3])
h = tf.nn.sigmoid(tf.matmul(x, W1) + b1)
W2 = tf.random.normal([3,2])
b2 = tf.zeros([2])
y = tf.nn.softmax(tf.matmul(h, W2) + b2)
print(y)
```
2) **Wrap forward in @tf.function**
```python
@tf.function
def forward(x, W1, b1, W2, b2):
    h = tf.nn.sigmoid(tf.matmul(x, W1) + b1)
    return tf.nn.softmax(tf.matmul(h, W2) + b2)
```

Run these in a live kernel to see eager vs compiled behavior.

### Figures: remaining visuals
<p align='left'><img src="./figure/figure2.9.png" width="48%"> <img src="./figure/figure2.10.png" width="48%"></p>
<p align='left'><img src="./figure/figure2.11.png" width="48%"> <img src="./figure/figure2.12.png" width="48%"></p>
<p align='left'><img src="./figure/figure2.13.png" width="60%"></p>
_These figures correspond to additional diagrams and sample outputs found in Chapter 2 (kernel execution, profiling view, convolution examples, etc.)._

## Chapter 2 — Summary (key takeaways)
- TensorFlow 2 favors eager execution for developer ergonomics while enabling graph compilation via `@tf.function` for performance.
- Core primitives: `tf.Variable` (mutable state), `tf.Tensor` (immutable arrays), `tf.Operation` (computational ops).
- Common NN ops (matmul, conv, pooling) are optimized on CPU/GPU/TPU.
- AutoGraph reduces boilerplate but requires careful use to avoid tracing overhead or big graphs.

If you want, I can now:
1. Save this as a `.ipynb` file in `/mnt/data/Chapter2_TensorFlow2.ipynb` for direct download, or
2. Add more runnable cells and unit tests, or
3. Convert everything into a markdown file for README.

Which of the above would you like next?